In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import cv2

In [2]:
df = pd.read_csv('AC_SOC2_1DLC_resnet50_social_behavior_allMay27shuffle1_250000.csv', header = [1, 2])

In [2]:
def prepare_coordinates(direc, padding = 40):

    """
    Designed to extract the coordinates that were created with the 'define coordinates' function
    in the terminal and import them into appropriate variables
    for jupyter notebook analysis
    """
    
    import numpy as np
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    
    ## will want to start things off by loading in the coordinates drawn (this should be done beforehand)

    left_side = np.loadtxt(direc + 'left_side')
    x_chamber = np.loadtxt(direc + 'x_chamber')
    y_chamber = np.loadtxt(direc + 'y_chamber')
    right_side = np.loadtxt(direc + 'right_side')
    middle = np.loadtxt(direc + 'middle')
    
    ## calculate the padded interaction zone, that is 40 pixels larger in every direction (than the base of the chamber)

    x_outer = Polygon(Polygon(x_chamber).buffer(padding).exterior).exterior.coords.xy
    y_outer = Polygon(Polygon(y_chamber).buffer(padding).exterior).exterior.coords.xy
    x_center = np.mean(x_outer[0]), np.mean(x_outer[1])
    y_center = np.mean(y_outer[0]), np.mean(y_outer[1])
    
    x_zone = Polygon(Polygon(x_chamber).buffer(padding).exterior)
    y_zone = Polygon(Polygon(y_chamber).buffer(padding).exterior)
    
    possible_places = {'x_zone': x_zone, 'y_zone': y_zone, 'left_side': left_side, 'middle': middle, 'right_side': right_side}
    extra_coords = {'x_outer': x_outer, 'x_center': x_center, 'y_outer': y_outer, 'y_center': y_center, 'x_chamber': x_chamber, 'y_chamber': y_chamber}
    
    return possible_places, extra_coords

In [4]:
possible_places, extra_coords = prepare_coordinates('coordinates\\')

In [4]:
def check_orientation(df):
    
    ## function that finds the distance between nose, and ears, from center of zone.
    ## returns either 'oriented' or 'not oriented' based on whether the nose is closest to center
    
    orientation = ['not_oriented'] * len(df)
    
    z = -1
    
    for index, val in df.iterrows():
        z = z + 1
        
        dist_to_x = np.sqrt(((x_center[0] - df['nose']['x'].loc[index])**2) + ((x_center[1] - df['nose']['y'].loc[index])**2))
        dist_to_y = np.sqrt(((y_center[0] - df['nose']['x'].loc[index])**2) + ((y_center[1] - df['nose']['y'].loc[index])**2))
        
        if dist_to_x > dist_to_y:        
            distance_to_nose = np.sqrt(((y_center[0] - df['nose']['x'].loc[index])**2) + ((y_center[1] - df['nose']['y'].loc[index])**2))
            distance_to_l_ear = np.sqrt(((y_center[0] - df['left ear']['x'].loc[index])**2) + ((y_center[1] - df['left ear']['y'].loc[index])**2))
            distance_to_r_ear = np.sqrt(((y_center[0] - df['right ear']['x'].loc[index])**2) + ((y_center[1] - df['right ear']['y'].loc[index])**2))
        elif dist_to_x < dist_to_y:
            distance_to_nose = np.sqrt(((x_center[0] - df['nose']['x'].loc[index])**2) + ((x_center[1] - df['nose']['y'].loc[index])**2))
            distance_to_l_ear = np.sqrt(((x_center[0] - df['left ear']['x'].loc[index])**2) + ((x_center[1] - df['left ear']['y'].loc[index])**2))
            distance_to_r_ear = np.sqrt(((x_center[0] - df['right ear']['x'].loc[index])**2) + ((x_center[1] - df['right ear']['y'].loc[index])**2))

        ## just look for what distance is the shortest. if that distance is nose, oriented, otherwise not
        
        if distance_to_nose < distance_to_l_ear:
            if distance_to_nose < distance_to_r_ear:
                orientation[z] = 'oriented'
            elif distance_to_nose > distance_to_r_ear:
                orientation[z] = 'not_oriented'              
        elif distance_to_nose > distance_to_l_ear:
            orientation[z] = 'not_oriented'
        elif distance_to_nose > distance_to_r_ear:
            orientation[z] = 'not_oriented'
            
    return orientation

In [3]:
def check_orientation_single(df, index_loc):
        
        orientation = 'not_oriented'
    
        dist_to_x = np.sqrt(((x_center[0] - df['nose']['x'].loc[index_loc])**2) + ((x_center[1] - df['nose']['y'].loc[index_loc])**2))
        dist_to_y = np.sqrt(((y_center[0] - df['nose']['x'].loc[index_loc])**2) + ((y_center[1] - df['nose']['y'].loc[index_loc])**2))
        
        if dist_to_x > dist_to_y:        
            distance_to_nose = np.sqrt(((y_center[0] - df['nose']['x'].loc[index_loc])**2) + ((y_center[1] - df['nose']['y'].loc[index_loc])**2))
            distance_to_l_ear = np.sqrt(((y_center[0] - df['left ear']['x'].loc[index_loc])**2) + ((y_center[1] - df['left ear']['y'].loc[index_loc])**2))
            distance_to_r_ear = np.sqrt(((y_center[0] - df['right ear']['x'].loc[index_loc])**2) + ((y_center[1] - df['right ear']['y'].loc[index_loc])**2))
        elif dist_to_x < dist_to_y:
            distance_to_nose = np.sqrt(((x_center[0] - df['nose']['x'].loc[index_loc])**2) + ((x_center[1] - df['nose']['y'].loc[index_loc])**2))
            distance_to_l_ear = np.sqrt(((x_center[0] - df['left ear']['x'].loc[index_loc])**2) + ((x_center[1] - df['left ear']['y'].loc[index_loc])**2))
            distance_to_r_ear = np.sqrt(((x_center[0] - df['right ear']['x'].loc[index_loc])**2) + ((x_center[1] - df['right ear']['y'].loc[index_loc])**2))
            
        if distance_to_nose == np.min([distance_to_nose, distance_to_l_ear, distance_to_r_ear]):
            orientation = 'oriented'
            
        return orientation

In [5]:
check_orientation_single(df, 560)

NameError: name 'x_center' is not defined